In [ ]:
from IPython.display import HTML
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
from skimage.transform import resize
import imageio
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import warnings

In [10]:
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis("off")
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [12]:
warnings.filterwarnings("ignore")
source_image = imageio.imread("demo/images/01.png")
reader = imageio.get_reader("demo/videos/0.mp4")
source_image = resize(source_image, (256, 256))[..., :3]
fps = reader.get_meta_data()["fps"]
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [ ]:
"""
HTML(display(source_image, driving_video).to_html5_video())
generator, kp_detector = load_checkpoints(
    config_path="config/vox-256.yaml", checkpoint_path="vox-cpk.pth.tar"
)
"""
generator, kp_detector = load_checkpoints(
    config_path="config/vox-adv-256.yaml", checkpoint_path="vox-adv-cpk.pth.tar"
)
predictions = make_animation(
    source_image, driving_video, generator, kp_detector, relative=True
)
imageio.mimsave(
    "generated.mp4", [img_as_ubyte(frame) for frame in predictions], fps=fps
)
HTML(display(source_image, driving_video, predictions).to_html5_video())